In [154]:
%reload_ext autoreload
%autoreload 2

import pandas as pd
from scipy.stats import f_oneway

data  = pd.read_csv('data/julia2018/derivatives/connectivity/dosenbach2010_partial_correlation.csv')

region2network = data.groupby('region_src')['network_src'].first().to_dict()

data['group'] = data['subject'].apply(lambda x: 1 if 'AVGP' in x else 0)

data['connectivity_name'] = data['network_src'] + '_' + data['network_dst']
data['connectivity'] = data['connectivity'].abs()

# convert data to wide format
data = data.pivot_table(index=['subject', 'group'], columns='connectivity_name', values='connectivity', aggfunc='mean')

cols = data.columns

cols = [set(c.split('_')) for c in cols]
cols = [c for c in cols if len(c) == 2]

cc = []
[cc.append(x) for x in cols if x not in cc]
cc = [list(c) for c in cc]

cols = [f'{c[0]}_{c[1]}' for c in cc]
cols = sorted(cols)

data = data[cols].reset_index()

from statsmodels.formula.api import ols

ols_model = ols('group ~ 1 + ' + '+'.join( cols), data=data).fit()
# ols_model.summary()

from statsmodels.stats.anova import anova_lm
anova_lm(ols_model)

,df,sum_sq,mean_sq,F,PR(>F)
CON_CER,1.0,0.000885,0.000885,0.005186,0.943483
CON_FPN,1.0,0.463407,0.463407,2.716250,0.118825
CON_VIS,1.0,0.725573,0.725573,4.252931,0.055810
DMN_CER,1.0,0.452708,0.452708,2.653538,0.122847
DMN_CON,1.0,1.607400,1.607400,9.421742,0.007334
DMN_FPN,1.0,0.111754,0.111754,0.655046,0.430187
DMN_SMN,1.0,0.652621,0.652621,3.825323,0.068176
DMN_VIS,1.0,0.707267,0.707267,4.145634,0.058647
FPN_CER,1.0,0.097264,0.097264,0.570110,0.461186
SMN_CER,1.0,0.029435,0.029435,0.172530,0.683390


In [153]:
avgp = data.query('subject.str.contains("AVGP")')
nvgp = data.query('subject.str.contains("NVGP")')

for col in data.columns:
    if '_' in col:
        f = f_oneway(avgp[col].tolist(), nvgp[col].tolist())
        if f.pvalue < 0.05:
            print(col, f)
